In [1]:
import gymnasium as gym
from x_mpsc import envs
import matplotlib.pyplot as plt


from safe_control_gym.controllers.safe_explorer.safe_ppo import SafeExplorerPPO

from pets_torch.utils.wrappers import VideoWrapper
from train_safe_exp import *

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf

#EXP_DIR = "../data/outputs/SimplePendulum-v0/safe_explorer_ppo/train/2023-01-12_16-12-43/"
EXP_DIR = "../data/outputs/TwoLinkArm-v0/safe_explorer_ppo/train/2023-01-12_16-56-19"

cfg = OmegaConf.load(os.path.join(EXP_DIR, ".hydra", "config.yaml"))

cfg.algo

{'name': 'safe_explorer_ppo', 'config': {'hidden_dim': 64, 'norm_obs': False, 'norm_reward': False, 'clip_obs': 10.0, 'clip_reward': 10.0, 'pretraining': False, 'pretrained': '/home2/tom.haider/Code/x_mpsc/data/outputs/TwoLinkArm-v0/safe_explorer_ppo/pretrain/2023-01-12_15-46-03/checkpoints/model_latest.pt', 'constraint_hidden_dim': 10, 'constraint_lr': 0.0001, 'constraint_batch_size': 256, 'constraint_steps_per_epoch': 6000, 'constraint_epochs': 25, 'constraint_eval_steps': 1500, 'constraint_eval_interval': 5, 'constraint_buffer_size': 1000000, 'constraint_slack': 0.05, 'gamma': 0.99, 'use_gae': False, 'gae_lambda': 0.95, 'use_clipped_value': False, 'clip_param': 0.2, 'target_kl': 0.01, 'entropy_coef': 0.01, 'opt_epochs': 10, 'mini_batch_size': 64, 'actor_lr': 0.0003, 'critic_lr': 0.001, 'max_grad_norm': 0.5, 'max_env_steps': 1000000, 'num_workers': 1, 'rollout_batch_size': 4, 'rollout_steps': 100, 'deque_size': 10, 'eval_batch_size': 10, 'log_interval': 1000, 'save_interval': 1000, '

In [3]:
def make_env(task, task_config, seed=42, record_videos=False):
    env = gym.make(task)
    env.seed(seed)

    constraints = []
    for c in task_config.constraints:
        obj = globals().get(c["cls_name"])(**c["kwargs"])
        constraints.append(obj)

    env = SCGWrapper(env, constraints=constraints)
    if record_videos:
        env = VideoWrapper(env, episode_trigger=lambda x:True, video_folder=f"./tmp/{cfg.task.name}/")
    return env


In [4]:


env_func = partial(make_env, task=cfg.task.name, task_config=cfg.task.config, record_videos=True)
agent = SafeExplorerPPO(
    env_func=env_func,
    training=False,
    device=cfg.device,
    **cfg.algo.config
)

agent.load(os.path.join(EXP_DIR, "checkpoints", "model_latest.pt"))
agent.reset()

pygame 2.1.2 (SDL 2.0.16, Python 3.8.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
WARNING! Video folder exists. Might overwrite exisiting videos.


/venvs/xmpsc/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [5]:
eval_results = agent.run(n_episodes=10, render=False)

Failed to create secure directory (/run/user/384214913/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5233:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM default
/venvs/xmpsc/lib/python3.8/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [19]:
ex_obs = np.array(eval_results["ep_observations"][-2])[8]


array([ 0.02776747,  0.537329  , -0.12959732,  0.789296  ,  0.48824033,
        0.8727092 ,  0.6082973 ,  0.7937093 ,  2.709683  , -6.016095  ],
      dtype=float32)

In [10]:
eval_results["constraint_violation"]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [23]:
env = env_func(record_videos=False)
env

/venvs/xmpsc/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


<SCGWrapper<TimeLimit<TwoLinkArmEnv<TwoLinkArm-v0>>>>

In [29]:
c_values = np.array([c.get_value(ex_obs) for c in env.constraints])
c_violation = any(c_values > 0)
c_violation

True

In [ ]:
c